# ETL - Ventas

## 1. Extracción.

### 1.1. importamos el DataFrame ventas.csv

In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns

sns.set_theme(style="whitegrid")

# Ruta al archivo Parquet
ruta_archivo = Path("data/raw/tecnoStore/ventas.csv")

try:
    if not ruta_archivo.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {ruta_archivo.resolve()}")
    
    df = pd.read_csv(ruta_archivo)

    print(" Extracción correcta")
    print(f" Registros cargados: {len(df)}")
    print(f" Columnas: {len(df.columns)}")


except FileNotFoundError as e:
    print(f"❌ Error: {e}")
except Exception as e:
    print(f"⚠️ Error inesperado: {e}")

 Extracción correcta
 Registros cargados: 224678
 Columnas: 8


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224678 entries, 0 to 224677
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   transaction_id   224678 non-null  int64  
 1   fecha            224678 non-null  object 
 2   cliente_id       224678 non-null  int64  
 3   producto_id      224678 non-null  int64  
 4   canal_id         224678 non-null  int64  
 5   sucursal_id      146738 non-null  float64
 6   cantidad         224678 non-null  int64  
 7   monto_venta_usd  224678 non-null  float64
dtypes: float64(2), int64(5), object(1)
memory usage: 13.7+ MB


In [3]:
df.head()

,transaction_id,fecha,cliente_id,producto_id,canal_id,sucursal_id,cantidad,monto_venta_usd
0,1,2018-01-01,2012,5,2,17.0,1,219.02
1,2,2018-01-01,3415,15,2,7.0,1,77.72
2,3,2018-01-01,1093,17,2,15.0,1,100.28
3,4,2018-01-01,2567,10,2,9.0,1,1051.77
4,5,2018-01-01,3274,11,2,14.0,1,800.92


## 2. Transformación

### 2.1. Busqueda de Duplicados

In [6]:
# Filtrar duplicados (todas las columnas)
df_duplicados = df[df.duplicated()]
print(df_duplicados)
# Filtrar duplicados (columnas críticas)
df_duplicados_id = df[df.duplicated(subset=["transaction_id"], keep=False)]
print("transaction_id:\n", df_duplicados_id)


Empty DataFrame
Columns: [transaction_id, fecha, cliente_id, producto_id, canal_id, sucursal_id, cantidad, monto_venta_usd]
Index: []
transaction_id:
 Empty DataFrame
Columns: [transaction_id, fecha, cliente_id, producto_id, canal_id, sucursal_id, cantidad, monto_venta_usd]
Index: []


### 2.2. Busqueda Valores Null (NaN)

In [7]:
print(df.isnull().sum()) # Muestra valores nulos (valor cero no es nulo)

transaction_id         0
fecha                  0
cliente_id             0
producto_id            0
canal_id               0
sucursal_id        77940
cantidad               0
monto_venta_usd        0
dtype: int64


Esos NaN en sucursal_id podrian ser ventas online, asi que lo verificaremos

In [8]:
ventas_online = df[df["sucursal_id"].isna() & (df["canal_id"] == 1)]
print(ventas_online.head())

# Contar valores nulos en la columna 'sucursal_id'
nulos_sucursal = df["sucursal_id"].isna().sum()
print("Nulos en sucursal_id: ", nulos_sucursal)
print("Ventas Online:     -> ", ventas_online.shape[0])


    transaction_id       fecha  cliente_id  producto_id  canal_id  \
9               10  2018-01-01         709           12         1   
10              11  2018-01-01        2218           16         1   
22              23  2018-01-01        4307           20         1   
29              30  2018-01-01        2708            5         1   
39              40  2018-01-01        3559            4         1   

    sucursal_id  cantidad  monto_venta_usd  
9           NaN         2          1011.32  
10          NaN         1           571.51  
22          NaN         1           201.01  
29          NaN         1           215.70  
39          NaN         2           294.85  
Nulos en sucursal_id:  77940
Ventas Online:     ->  77940


In [9]:
# Reemplazar NaN en sucursal_id por 0 que haremos que sea para ventas online
df.loc[df["sucursal_id"].isna(), "sucursal_id"] = 0

print("Registros Modificados: ", (df["sucursal_id"] == 0).sum())

Registros Modificados:  77940


## 3. Carga

### 3.1. Guardamos el DF transformado y limpio.

In [10]:
import os

filename = "data/processed/tecnoStore/fact_ventas.parquet"

try:
    df.to_parquet(filename, index=False)
    if os.path.exists(filename):
        print(f"✔️ Dataset guardado con éxito en: {filename}")
    else:
        print("⚠️ No se encontró el archivo después de guardar.")
except Exception as e:
    print(f"❌ Error al guardar el dataset: {e}")

✔️ Dataset guardado con éxito en: data/processed/tecnoStore/fact_ventas.parquet


### 3.2. Generamos la Tabla Calendar

In [11]:
import pandas as pd

fechas = pd.date_range(
    start=df["fecha"].min(),
    end=df["fecha"].max(),
    freq="D"
)

dias = {
    "Monday":"Lunes","Tuesday":"Martes","Wednesday":"Miércoles",
    "Thursday":"Jueves","Friday":"Viernes","Saturday":"Sábado","Sunday":"Domingo"
}
meses = {
    "January":"Enero","February":"Febrero","March":"Marzo","April":"Abril",
    "May":"Mayo","June":"Junio","July":"Julio","August":"Agosto",
    "September":"Septiembre","October":"Octubre","November":"Noviembre","December":"Diciembre"
}

calendario = pd.DataFrame({"fecha": fechas})
calendario["año"] = calendario["fecha"].dt.year
calendario["mes"] = calendario["fecha"].dt.month
calendario["nombre_mes"] = calendario["fecha"].dt.strftime("%B").map(meses)
calendario["trimestre"] = "T" + calendario["fecha"].dt.quarter.astype(str)
calendario["semana_año"] = calendario["fecha"].dt.isocalendar().week
calendario["día"] = calendario["fecha"].dt.day
calendario["nombre_día"] = calendario["fecha"].dt.strftime("%A").map(dias)


In [12]:
calendario.head()

,fecha,año,mes,nombre_mes,trimestre,semana_año,día,nombre_día
0,2018-01-01,2018,1,Enero,T1,1,1,Lunes
1,2018-01-02,2018,1,Enero,T1,1,2,Martes
2,2018-01-03,2018,1,Enero,T1,1,3,Miércoles
3,2018-01-04,2018,1,Enero,T1,1,4,Jueves
4,2018-01-05,2018,1,Enero,T1,1,5,Viernes


### 3.3. Guardamos el DF Calendario.

In [13]:
import os

filename = "data/processed/tecnoStore/dim_calendario.parquet"

try:
    calendario.to_parquet(filename, index=False)
    if os.path.exists(filename):
        print(f"✔️ Dataset guardado con éxito en: {filename}")
    else:
        print("⚠️ No se encontró el archivo después de guardar.")
except Exception as e:
    print(f"❌ Error al guardar el dataset: {e}")

✔️ Dataset guardado con éxito en: data/processed/tecnoStore/dim_calendario.parquet
